In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고가 뜨지 않도록..
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 출력 창 청소
from IPython.display import clear_output

import time


# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 딥러닝
import tensorflow as tf

# 딥러닝 모델 구조를 정의하는 것
from tensorflow.keras.models import Sequential
# 층 모델(은닉층과 출력층)
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 것
from tensorflow.keras.layers import Activation

# 데이터 표준화
from sklearn.preprocessing import StandardScaler
# 문자열을 숫자로 변환
from sklearn.preprocessing import LabelEncoder
# 원핫인코딩
from tensorflow.keras.utils import to_categorical

# 학습데이터와 검증데이터로 나눈다
from sklearn.model_selection import train_test_split

# K폴드 교차검증
from sklearn.model_selection import KFold


# GPU 할당
gpus = tf.config.experimental.list_physical_devices('GPU')
# GPU가 있다면..
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)


In [2]:
# 랜덤 시드 설정
np.random.seed(1)
tf.random.set_seed(1)

In [3]:
df1 = pd.read_csv('data/sonar.csv', header=None)
df1

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


In [4]:
# 입력과 결과로 나눈다.
X = df1.drop(60, axis=1)
y = df1[60]

display(X)
display(y)

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0203,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0051,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0155,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0042,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048


0      R
1      R
2      R
3      R
4      R
      ..
203    M
204    M
205    M
206    M
207    M
Name: 60, Length: 208, dtype: object

In [5]:
# 문자열 데이터를 숫자로 변환한다.
encoder1 = LabelEncoder()
encoder1.fit(y)
y = encoder1.transform(y)
y


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [6]:
# 데이터 표준화
scaler1 = StandardScaler()
scaler1.fit(X)
X = scaler1.transform(X)
X

array([[-0.39955135, -0.04064823, -0.02692565, ...,  0.06987027,
         0.17167808, -0.65894689],
       [ 0.70353822,  0.42163039,  1.05561832, ..., -0.47240644,
        -0.44455424, -0.41985233],
       [-0.12922901,  0.60106749,  1.72340448, ...,  1.30935987,
         0.25276128,  0.25758223],
       ...,
       [ 1.00438083,  0.16007801, -0.67384349, ...,  0.90652575,
        -0.03913824, -0.67887143],
       [ 0.04953255, -0.09539176,  0.13480381, ..., -0.00759783,
        -0.70402047, -0.34015415],
       [-0.13794908, -0.06497869, -0.78861924, ..., -0.6738235 ,
        -0.29860448,  0.99479044]])

### 비율 설정
- 주어진 데이터를 랜덤하게 섞은 후 주어진 비율만큼 학습셋과 검증셋을 나눠 학습과 평가를 실시한다.

In [7]:
# 신경망 구조를 설계한다.
model = Sequential()

a1 = X.shape[1]
model.add(Dense(24, input_dim=a1))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [8]:
# 학습 모델을 생성한다
model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [10]:
# 학습한다.
# validation_split : 검증 데이터 비율
model.fit(X, y, epochs=200, batch_size=100, 
          validation_split=0.3)


Epoch 1/200
2/2 [==============================] - 1s 135ms/step - loss: 0.6830 - accuracy: 0.6138 - val_loss: 0.6876 - val_accuracy: 0.4286
Epoch 2/200
2/2 [==============================] - 0s 26ms/step - loss: 0.6593 - accuracy: 0.6483 - val_loss: 0.7038 - val_accuracy: 0.4127
Epoch 3/200
2/2 [==============================] - 0s 25ms/step - loss: 0.6382 - accuracy: 0.6966 - val_loss: 0.7197 - val_accuracy: 0.3333
Epoch 4/200
2/2 [==============================] - 0s 24ms/step - loss: 0.6193 - accuracy: 0.7034 - val_loss: 0.7346 - val_accuracy: 0.3016
Epoch 5/200
2/2 [==============================] - 0s 22ms/step - loss: 0.6031 - accuracy: 0.7172 - val_loss: 0.7488 - val_accuracy: 0.2857
Epoch 6/200
2/2 [==============================] - 0s 23ms/step - loss: 0.5870 - accuracy: 0.7517 - val_loss: 0.7624 - val_accuracy: 0.2381
Epoch 7/200
2/2 [==============================] - 0s 23ms/step - loss: 0.5732 - accuracy: 0.7586 - val_loss: 0.7750 - val_accuracy: 0.2222
Epoch 8/200
2/2 [==

2/2 [==============================] - 0s 30ms/step - loss: 0.1132 - accuracy: 0.9862 - val_loss: 1.8866 - val_accuracy: 0.2540
Epoch 60/200
2/2 [==============================] - 0s 30ms/step - loss: 0.1094 - accuracy: 0.9862 - val_loss: 1.9113 - val_accuracy: 0.2540
Epoch 61/200
2/2 [==============================] - 0s 27ms/step - loss: 0.1057 - accuracy: 0.9931 - val_loss: 1.9364 - val_accuracy: 0.2540
Epoch 62/200
2/2 [==============================] - 0s 29ms/step - loss: 0.1021 - accuracy: 0.9931 - val_loss: 1.9616 - val_accuracy: 0.2540
Epoch 63/200
2/2 [==============================] - 0s 27ms/step - loss: 0.0986 - accuracy: 1.0000 - val_loss: 1.9866 - val_accuracy: 0.2540
Epoch 64/200
2/2 [==============================] - 0s 27ms/step - loss: 0.0953 - accuracy: 1.0000 - val_loss: 2.0100 - val_accuracy: 0.2540
Epoch 65/200
2/2 [==============================] - 0s 28ms/step - loss: 0.0921 - accuracy: 1.0000 - val_loss: 2.0334 - val_accuracy: 0.2540
Epoch 66/200
2/2 [========

Epoch 117/200
2/2 [==============================] - 0s 25ms/step - loss: 0.0192 - accuracy: 1.0000 - val_loss: 3.1663 - val_accuracy: 0.2222
Epoch 118/200
2/2 [==============================] - 0s 25ms/step - loss: 0.0188 - accuracy: 1.0000 - val_loss: 3.1821 - val_accuracy: 0.2222
Epoch 119/200
2/2 [==============================] - 0s 25ms/step - loss: 0.0184 - accuracy: 1.0000 - val_loss: 3.1975 - val_accuracy: 0.2222
Epoch 120/200
2/2 [==============================] - 0s 25ms/step - loss: 0.0179 - accuracy: 1.0000 - val_loss: 3.2135 - val_accuracy: 0.2222
Epoch 121/200
2/2 [==============================] - 0s 25ms/step - loss: 0.0176 - accuracy: 1.0000 - val_loss: 3.2294 - val_accuracy: 0.2222
Epoch 122/200
2/2 [==============================] - 0s 25ms/step - loss: 0.0172 - accuracy: 1.0000 - val_loss: 3.2450 - val_accuracy: 0.2222
Epoch 123/200
2/2 [==============================] - 0s 26ms/step - loss: 0.0168 - accuracy: 1.0000 - val_loss: 3.2608 - val_accuracy: 0.2222
Epoch 

Epoch 175/200
2/2 [==============================] - 0s 25ms/step - loss: 0.0066 - accuracy: 1.0000 - val_loss: 3.9682 - val_accuracy: 0.2381
Epoch 176/200
2/2 [==============================] - 0s 24ms/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 3.9798 - val_accuracy: 0.2381
Epoch 177/200
2/2 [==============================] - 0s 26ms/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 3.9913 - val_accuracy: 0.2381
Epoch 178/200
2/2 [==============================] - 0s 29ms/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 4.0028 - val_accuracy: 0.2381
Epoch 179/200
2/2 [==============================] - 0s 26ms/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 4.0143 - val_accuracy: 0.2381
Epoch 180/200
2/2 [==============================] - 0s 27ms/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 4.0251 - val_accuracy: 0.2381
Epoch 181/200
2/2 [==============================] - 0s 28ms/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 4.0355 - val_accuracy: 0.2540
Epoch 

### 검증용 데이터가 있을 때 (split 안해도 이미 나눠져 있을 때)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                   random_state=1)


In [12]:
# 신경망 구조를 설계한다.
model = Sequential()

a1 = X.shape[1]
model.add(Dense(24, input_dim=a1))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))


In [13]:
# 학습 모델을 생성한다.
model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=['accuracy'])


In [14]:
# 학습한다.
# validation_split : 검증 데이터 비율
model.fit(X_train, y_train, epochs=200, batch_size=100, 
          validation_data=(X_test, y_test))


Epoch 1/200
2/2 [==============================] - 0s 115ms/step - loss: 0.7220 - accuracy: 0.5586 - val_loss: 0.7525 - val_accuracy: 0.5556
Epoch 2/200
2/2 [==============================] - 0s 28ms/step - loss: 0.6989 - accuracy: 0.6138 - val_loss: 0.7395 - val_accuracy: 0.6349
Epoch 3/200
2/2 [==============================] - 0s 29ms/step - loss: 0.6810 - accuracy: 0.6552 - val_loss: 0.7281 - val_accuracy: 0.6508
Epoch 4/200
2/2 [==============================] - 0s 31ms/step - loss: 0.6650 - accuracy: 0.6759 - val_loss: 0.7175 - val_accuracy: 0.6508
Epoch 5/200
2/2 [==============================] - 0s 30ms/step - loss: 0.6496 - accuracy: 0.6966 - val_loss: 0.7075 - val_accuracy: 0.6825
Epoch 6/200
2/2 [==============================] - 0s 28ms/step - loss: 0.6360 - accuracy: 0.7103 - val_loss: 0.6983 - val_accuracy: 0.6667
Epoch 7/200
2/2 [==============================] - 0s 28ms/step - loss: 0.6228 - accuracy: 0.7517 - val_loss: 0.6902 - val_accuracy: 0.6667
Epoch 8/200
2/2 [==

Epoch 59/200
2/2 [==============================] - 0s 25ms/step - loss: 0.1810 - accuracy: 0.9655 - val_loss: 0.4830 - val_accuracy: 0.7937
Epoch 60/200
2/2 [==============================] - 0s 24ms/step - loss: 0.1758 - accuracy: 0.9655 - val_loss: 0.4830 - val_accuracy: 0.7937
Epoch 61/200
2/2 [==============================] - 0s 25ms/step - loss: 0.1711 - accuracy: 0.9655 - val_loss: 0.4833 - val_accuracy: 0.7937
Epoch 62/200
2/2 [==============================] - 0s 26ms/step - loss: 0.1663 - accuracy: 0.9724 - val_loss: 0.4835 - val_accuracy: 0.8095
Epoch 63/200
2/2 [==============================] - 0s 26ms/step - loss: 0.1616 - accuracy: 0.9724 - val_loss: 0.4839 - val_accuracy: 0.8095
Epoch 64/200
2/2 [==============================] - 0s 27ms/step - loss: 0.1572 - accuracy: 0.9724 - val_loss: 0.4846 - val_accuracy: 0.8095
Epoch 65/200
2/2 [==============================] - 0s 26ms/step - loss: 0.1532 - accuracy: 0.9793 - val_loss: 0.4853 - val_accuracy: 0.8095
Epoch 66/200


Epoch 117/200
2/2 [==============================] - 0s 26ms/step - loss: 0.0402 - accuracy: 1.0000 - val_loss: 0.5594 - val_accuracy: 0.8413
Epoch 118/200
2/2 [==============================] - 0s 26ms/step - loss: 0.0392 - accuracy: 1.0000 - val_loss: 0.5609 - val_accuracy: 0.8413
Epoch 119/200
2/2 [==============================] - 0s 26ms/step - loss: 0.0383 - accuracy: 1.0000 - val_loss: 0.5625 - val_accuracy: 0.8413
Epoch 120/200
2/2 [==============================] - 0s 25ms/step - loss: 0.0373 - accuracy: 1.0000 - val_loss: 0.5642 - val_accuracy: 0.8413
Epoch 121/200
2/2 [==============================] - 0s 30ms/step - loss: 0.0365 - accuracy: 1.0000 - val_loss: 0.5659 - val_accuracy: 0.8413
Epoch 122/200
2/2 [==============================] - 0s 29ms/step - loss: 0.0356 - accuracy: 1.0000 - val_loss: 0.5674 - val_accuracy: 0.8413
Epoch 123/200
2/2 [==============================] - 0s 28ms/step - loss: 0.0348 - accuracy: 1.0000 - val_loss: 0.5687 - val_accuracy: 0.8413
Epoch 

Epoch 175/200
2/2 [==============================] - 0s 37ms/step - loss: 0.0129 - accuracy: 1.0000 - val_loss: 0.6574 - val_accuracy: 0.8095
Epoch 176/200
2/2 [==============================] - 0s 37ms/step - loss: 0.0127 - accuracy: 1.0000 - val_loss: 0.6591 - val_accuracy: 0.8095
Epoch 177/200
2/2 [==============================] - 0s 34ms/step - loss: 0.0125 - accuracy: 1.0000 - val_loss: 0.6610 - val_accuracy: 0.8095
Epoch 178/200
2/2 [==============================] - 0s 31ms/step - loss: 0.0123 - accuracy: 1.0000 - val_loss: 0.6628 - val_accuracy: 0.8095
Epoch 179/200
2/2 [==============================] - 0s 28ms/step - loss: 0.0121 - accuracy: 1.0000 - val_loss: 0.6646 - val_accuracy: 0.8095
Epoch 180/200
2/2 [==============================] - 0s 27ms/step - loss: 0.0119 - accuracy: 1.0000 - val_loss: 0.6663 - val_accuracy: 0.8095
Epoch 181/200
2/2 [==============================] - 0s 27ms/step - loss: 0.0117 - accuracy: 1.0000 - val_loss: 0.6681 - val_accuracy: 0.8095
Epoch 